In [1]:
dir_path = "FinanceDB/Assets"
import os


In [2]:
import pandas as pd
import math
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [3]:
os.getcwd()

'/home/samar/Desktop/PROJECTS/Homer_3'

In [4]:
df_price = pd.read_csv(f'{dir_path}/corp_prices/RELIANCE_10yr_price_md.csv')
df_fed = pd.read_csv(f'{dir_path}/FRED/W/fed_assets.csv')

In [5]:
df_price.set_index('date', inplace=True)
df_price = df_price.drop('symbol',axis=1)

In [6]:
df_fed.set_index('date', inplace=True)
df_price = df_price.drop('Unnamed: 0', axis=1)
df_price = df_price.drop('dividends', axis=1)
df_price.head()

,low,volume,open,high,close,adjclose,splits,max_dv,close_dv,mov_1d,vol_1d
date,,,,,,,,,,,
1996-01-01,15.754989,48051995.0,15.859429,15.944529,15.917452,10.543346,0.0,0.012030,0.003659,NaN,NaN
1996-01-02,15.677626,77875009.0,15.878770,15.956133,15.793671,10.461358,0.0,0.017765,-0.005359,-0.007776,0.620640
1996-01-03,15.878770,96602936.0,16.052837,16.783918,15.913584,10.540785,0.0,0.057004,-0.008675,0.007592,0.240487
1996-01-04,15.553845,100099436.0,15.762726,15.813012,15.766594,10.443419,0.0,0.016663,0.000245,-0.009237,0.036195
1996-01-05,15.522900,76935930.0,15.704703,15.704703,15.658285,10.371678,0.0,0.011712,-0.002956,-0.006870,-0.231405


In [7]:
df_price = df_price.drop('splits', axis=1)
df_price.head()

,low,volume,open,high,close,adjclose,max_dv,close_dv,mov_1d,vol_1d
date,,,,,,,,,,
1996-01-01,15.754989,48051995.0,15.859429,15.944529,15.917452,10.543346,0.012030,0.003659,NaN,NaN
1996-01-02,15.677626,77875009.0,15.878770,15.956133,15.793671,10.461358,0.017765,-0.005359,-0.007776,0.620640
1996-01-03,15.878770,96602936.0,16.052837,16.783918,15.913584,10.540785,0.057004,-0.008675,0.007592,0.240487
1996-01-04,15.553845,100099436.0,15.762726,15.813012,15.766594,10.443419,0.016663,0.000245,-0.009237,0.036195
1996-01-05,15.522900,76935930.0,15.704703,15.704703,15.658285,10.371678,0.011712,-0.002956,-0.006870,-0.231405


In [8]:
df_price.head()

,low,volume,open,high,close,adjclose,max_dv,close_dv,mov_1d,vol_1d
date,,,,,,,,,,
1996-01-01,15.754989,48051995.0,15.859429,15.944529,15.917452,10.543346,0.012030,0.003659,NaN,NaN
1996-01-02,15.677626,77875009.0,15.878770,15.956133,15.793671,10.461358,0.017765,-0.005359,-0.007776,0.620640
1996-01-03,15.878770,96602936.0,16.052837,16.783918,15.913584,10.540785,0.057004,-0.008675,0.007592,0.240487
1996-01-04,15.553845,100099436.0,15.762726,15.813012,15.766594,10.443419,0.016663,0.000245,-0.009237,0.036195
1996-01-05,15.522900,76935930.0,15.704703,15.704703,15.658285,10.371678,0.011712,-0.002956,-0.006870,-0.231405


In [9]:
forecast_col = 'close'
forecast_out = int(math.ceil(0.01*len(df_price)))

In [10]:
dfp = df_price

In [11]:
dfp['label'] = dfp[forecast_col].shift(-forecast_out)

In [12]:
dfp['max_dv'] = dfp['max_dv'] * 100
dfp['mov_1d'] = dfp['mov_1d'] * 100
dfp['close_dv'] = dfp['close_dv'] * 100
dfp['vol_1d'] = dfp['vol_1d'] * 100

In [13]:

dfp.dropna(inplace=True)

In [14]:
dfp.replace([np.inf, -np.inf], np.nan, inplace=True)
dfp.head()
dfp.fillna(-9999, inplace=True)

In [15]:
X = np.array(dfp.drop(['label'],1))
y = np.array(dfp['label'])


In [16]:
X = preprocessing.scale(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [18]:
clf = svm.SVR()
clf.fit(X_train, y_train)

SVR()

In [19]:
conf = clf.score(X_test, y_test)
print(conf)

0.6077331737544558


In [20]:
clf = LinearRegression()
clf.fit(X_train, y_train)
con = clf.score(X_test, y_test)

In [21]:
print(con)

0.9581360449031081


In [22]:
for k in ['linear', 'poly', 'rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print(k, confidence)

linear 0.955923086643051
poly 0.5798967397694235
rbf 0.6077331737544558
sigmoid 0.620757532842123
